In [220]:
%pip install numpy>=1.24.1
%pip install ipython==8.10.0
%pip install pandas==2.0.3
%pip install causalnex

You should consider upgrading via the '/home/natalia/wsi/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/natalia/wsi/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
  Using cached pandas-2.0.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
causalnex 0.12.1 requires pandas<2.0,>=1.0, but you have pandas 2.0.3 which is incompatible.
You should consider upgrading via the '/home/natalia/wsi/venv/bin/python -m pip install --upgrade pip' command.
Note: you may nee

In [221]:
import pandas as pd

url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"

df = pd.read_csv(url)

print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [222]:
import warnings
from causalnex.structure import StructureModel

warnings.filterwarnings("ignore")  # silence warnings

sm = StructureModel()

In [223]:
sm.add_edges_from([
    ('pclass', 'cabin'), 
    ('pclass', 'fare')
])
sm.edges

OutEdgeView([('pclass', 'cabin'), ('pclass', 'fare')])

In [224]:
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE

viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)


viz.show("01_simple_plot.html")


01_simple_plot.html


In [225]:
drop_col = ['Ticket', 'Embarked', 'Cabin', 'Name', 'PassengerId']
data = df.drop(columns=drop_col)
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [226]:

import numpy as np

struct_data = data.copy()
non_numeric_columns = list(struct_data.select_dtypes(exclude=[np.number]).columns)

print(non_numeric_columns)

['Sex']


In [227]:

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in non_numeric_columns:
    struct_data[col] = le.fit_transform(struct_data[col])

struct_data.dropna(inplace=True)

struct_data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [228]:
from causalnex.structure.notears import from_pandas
sm = from_pandas(struct_data)

sm.remove_edges_below_threshold(0.8)

In [229]:
sm.remove_edge("Sex", "Fare")
sm.add_edge("Age", "Survived")
sm.add_edge("Sex", "Survived")
sm.remove_edge("Survived", "Age")
sm.remove_edge("Survived", "Pclass")
sm.remove_edge("Survived", "Fare")

In [230]:
sm = sm.get_largest_subgraph()
viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)

viz.toggle_physics(False)
viz.show("01_fully_connected.html")

01_fully_connected.html


In [231]:
import networkx as nx

nx.drawing.nx_pydot.write_dot(sm, 'graph.dot')

In [232]:
from causalnex.network import BayesianNetwork

bn = BayesianNetwork(sm)


discretised_data = struct_data.copy()

survived_map = {0: "Didn't Survive", 1: "Survived"}

discretised_data["Survived"] = discretised_data["Survived"].map(survived_map)

In [233]:
# Split 90% train and 10% test
from sklearn.model_selection import train_test_split

train, test = train_test_split(discretised_data, train_size=0.9, test_size=0.1, random_state=7)

In [234]:
bn = bn.fit_node_states(discretised_data)

In [235]:
bn = bn.fit_cpds(train, method="BayesianEstimator", bayes_prior="K2")

In [236]:
bn.cpds["Survived"]

Age            0.42            0.67            0.75       0.83             \
Sex                0         1     0         1     0    1     0         1   
Survived                                                                    
Didn't Survive   0.5  0.333333   0.5  0.333333  0.25  0.5   0.5  0.333333   
Survived         0.5  0.666667   0.5  0.666667  0.75  0.5   0.5  0.666667   

Age            0.92             ... 70.00           70.50           71.00  \
Sex                0         1  ...     0         1     0         1     0   
Survived                        ...                                         
Didn't Survive   0.5  0.333333  ...   0.5  0.666667   0.5  0.666667   0.5   
Survived         0.5  0.666667  ...   0.5  0.333333   0.5  0.333333   0.5   

Age                  74.00           80.00            
Sex                1     0         1     0         1  
Survived                                              
Didn't Survive  0.75   0.5  0.666667   0.5  0.333333  
Survived        0.25   0.5  0.333333   0.5  0.666667  

[2 rows x 176 columns]